Used to Query Gaia data, read in gaia data as well as c2d/GB data, and make some plots about the data

In [1]:
# Standard definitions needed for script
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import math
from astropy.table import Table
from astropy.io import fits
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from numpy.polynomial.polynomial import polyfit

In [2]:
# set-up for query to GAIA archive
from astroquery.gaia import Gaia
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving tables...
Parsing tables...
Done.
external.external.gaiadr2_geometric_distance
public.public.dual
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighbourhood
gaiadr1.gaiadr1.sdss_dr9_best_neighbour
gaiadr1.gaiadr1.sdss_dr9_neighbourhood
gaiadr1.gaiadr1.tmass_best_neighbour
gaiadr1.gaiadr1.tmass_neighbourhood
gaiadr1.gaiadr1.ucac4

In [3]:
# send request to GAIA archive
# the items in the box command are (corrdinate system, RA in deg, Dec in deg, box width in RA and Dec in deg)
query = "SELECT * \
FROM gaiadr2.gaia_source AS g, gaiadr2.tmass_best_neighbour AS tbest, gaiadr1.tmass_original_valid AS tmass \
WHERE g.source_id = tbest.source_id AND tbest.tmass_oid = tmass.tmass_oid \
AND CONTAINS(POINT('ICRS',g.ra,g.dec),BOX('ICRS',277.49,-1.00,5.,5.))=1 AND (parallax >=1.6 AND parallax_over_error >=3);"

job1 = Gaia.launch_job_async(query, dump_to_file=False)

Query finished.


In [4]:
# this retrieves the result from the query above and puts into a table with the name "p"
# The next line prints out the labels for all of the columns in the table
GTab = job1.get_results() 
print (GTab.colnames)
Gra = GTab["ra"]
Gdec = GTab["dec"]
Gpmra = GTab["pmra"]
Gpmra_e = GTab["pmra_error"]
Gpmdec = GTab["pmdec"]
Gpmdec_e = GTab["pmdec_error"]
Gpar = GTab["parallax"]
Gpar_sn = GTab["parallax_over_error"]
Gsepar = GTab["angular_distance"]
G2MASS = GTab["designation_2"]
print(" GAIA Table contains ",len(Gra)," sources")

['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_erro

In [5]:
infile = "SERAQU-YSOs.tbl"
GBYSO = Table.read(infile,format="ascii.ipac")
# print(GBYSO.colnames)
GBYSOra = GBYSO["ra"]
GBYSOdec = GBYSO["dec"]
GBYSO2MASS = GBYSO["id2mass"]
print("Read ",len(GBYSOra)," YSOs from GB YSO catalog")

Read  1442  YSOs from GB YSO catalog


In [6]:
gaia_list = []
for i in range(0,len(G2MASS)):
    gaia_list.append(G2MASS[i].decode('utf-8'))
    
GBYSO_match = np.empty(len(GBYSO2MASS),dtype=int); GBYSO_match.fill(-2)
for j in range(0,len(GBYSO2MASS)):
    if(GBYSO2MASS[j] in gaia_list):
        GBYSO_match[j] = gaia_list.index(GBYSO2MASS[j])
    else:
        GBYSO_match[j] = -2
good_GBYSO = GBYSO_match > -1
print(len(GBYSO_match[good_GBYSO]))
print(np.where(GBYSO_match > -1))

82
(array([ 150,  160,  194,  253,  279,  282,  309,  347,  348,  349,  360,
        371,  380,  382,  428,  433,  446,  495,  520,  690,  730,  734,
        762,  791,  793,  833,  844,  875,  939,  942,  961,  962,  964,
        974,  975,  978,  979,  982,  987,  992, 1036, 1042, 1043, 1047,
       1059, 1067, 1068, 1093, 1095, 1132, 1133, 1138, 1145, 1155, 1185,
       1213, 1226, 1227, 1233, 1253, 1255, 1258, 1269, 1276, 1286, 1288,
       1292, 1300, 1307, 1312, 1315, 1324, 1326, 1327, 1330, 1331, 1341,
       1345, 1356, 1388, 1402, 1406]),)


In [7]:
infile = "SERP_GB_star+dust.tbl"
GBSD = Table.read(infile,format="ascii.ipac")
# print(GBSD.colnames)
GBSDra = GBSD["ra"]
GBSDdec = GBSD["dec"]
GBSD2MASS = GBSD["id2mass"]
print("Read ",len(GBSDra)," YSOs from GB Star+Dust catalog")

GBSD_match = np.empty(len(GBSD2MASS),dtype=int); GBSD_match.fill(-2)
for j in range(0,len(GBSD2MASS)):
    if(GBSD2MASS[j] in gaia_list):
        GBSD_match[j] = gaia_list.index(GBSD2MASS[j])
    else:
        GBSD_match[j] = -2
good_GBSD = GBSD_match > -1
print(len(GBSD_match[good_GBSD]))
print(np.where(GBSD_match > -1))

Read  3725  YSOs from GB Star+Dust catalog
64
(array([ 744,  749,  850, 1311, 1336, 1401, 1470, 1764, 1793, 1880, 1907,
       2107, 2178, 2186, 2188, 2194, 2212, 2219, 2223, 2233, 2615, 2751,
       2752, 2902, 3084, 3119, 3127, 3129, 3133, 3136, 3138, 3170, 3175,
       3213, 3267, 3271, 3287, 3304, 3310, 3326, 3330, 3344, 3346, 3366,
       3369, 3372, 3375, 3380, 3406, 3417, 3435, 3465, 3478, 3480, 3488,
       3489, 3494, 3501, 3512, 3521, 3530, 3542, 3549, 3625]),)


In [8]:
infile = "SERP_c2d_star+dust.tbl"
c2dSD = Table.read(infile,format="ascii.ipac")
# print(GBSD.colnames)
c2dSDra = c2dSD["ra"]
c2dSDdec = c2dSD["dec"]
c2dSD2MASS = c2dSD["id2mass"]
print("Read ",len(c2dSDra)," YSOs from c2d Star+Dust catalog")

c2dSD_match = np.empty(len(c2dSD2MASS),dtype=int); c2dSD_match.fill(-2)
for j in range(0,len(c2dSD2MASS)):
    if(c2dSD2MASS[j] in gaia_list):
        c2dSD_match[j] = gaia_list.index(c2dSD2MASS[j])
    else:
        c2dSD_match[j] = -2
good_c2dSD = c2dSD_match > -1
print(len(c2dSD_match[good_c2dSD]))
print(np.where(c2dSD_match > -1))

Read  256  YSOs from c2d Star+Dust catalog
23
(array([  2,  29,  51,  54, 126, 128, 142, 147, 153, 156, 171, 175, 193,
       201, 214, 217, 219, 223, 224, 228, 235, 238, 251]),)


In [9]:
infile = "SERP_c2d_YSOs.tbl"
c2dYSO = Table.read(infile,format="ascii.ipac")
#print(c2dYSO.colnames)
c2dYSOra = c2dYSO["ra"]
c2dYSOdec = c2dYSO["dec"]
c2dYSO2MASS = c2dYSO["id2mass"]
c2dYSOobj = c2dYSO["object_type"]
print("Read ",len(c2dYSOra)," YSOs from c2d YSO catalog")

c2dYSO_match = np.empty(len(c2dYSO2MASS),dtype=int); c2dYSO_match.fill(-2)
for j in range(0,len(c2dYSO2MASS)):
    if(c2dYSO2MASS[j] in gaia_list):
        c2dYSO_match[j] = gaia_list.index(c2dYSO2MASS[j])
    else:
        c2dYSO_match[j] = -2
good_c2dYSO = c2dYSO_match > -1
print(len(c2dYSO_match[good_c2dYSO]))
print(np.where(c2dYSO_match > -1))

FileNotFoundError: [Errno 2] No such file or directory: 'SERP_c2d_YSOs.tbl'

In [ ]:
YSO_match = c2dYSO_match; YSO_match = np.append(YSO_match,GBYSO_match)
good_YSO = YSO_match > -1
print("Total number of GAIA matches to YSOs: ",len(YSO_match[good_YSO]))
SD_match = c2dSD_match; SD_match = np.append(SD_match,GBSD_match)
good_SD = SD_match > -1
print("Total number of GAIA matches to star+dust: ",len(SD_match[good_SD]))

In [ ]:
raYSO = np.zeros(len(YSO_match[good_YSO]))
decYSO = np.zeros(len(YSO_match[good_YSO]))
pmraYSO = np.zeros(len(YSO_match[good_YSO]))
pmdecYSO = np.zeros(len(YSO_match[good_YSO]))
pmra_eYSO = np.zeros(len(YSO_match[good_YSO]))
pmdec_eYSO = np.zeros(len(YSO_match[good_YSO]))
parYSO = np.zeros(len(YSO_match[good_YSO]))
par_snYSO = np.zeros(len(YSO_match[good_YSO]))
nyso = 0
for i in range(0,len(YSO_match)):
    if(YSO_match[i] > -1):
        j = YSO_match[i]
        if(1000./Gpar[j] > 350. and 1000/Gpar[j] < 550.):
            raYSO[nyso] = Gra[j]
            decYSO[nyso] = Gdec[j]
            pmraYSO[nyso] = Gpmra[j]
            pmdecYSO[nyso] = Gpmdec[j]
            parYSO[nyso] = Gpar[j]
            par_snYSO[nyso] = Gpar_sn[j]
            nyso = nyso + 1        
print("Number of YSO matches in the correct distance interval: ",nyso)
parsn10 = par_snYSO > 10.
parsn5 = par_snYSO > 5
parsn20 = par_snYSO > 20
print("Number of YSO with parallax > 5, 10, 20: ", len(par_snYSO[parsn5]),len(par_snYSO[parsn10]),len(par_snYSO[parsn20]))
plt.scatter(raYSO[parsn5],decYSO[parsn5],s=5)
plt.show()
plt.scatter(pmraYSO[parsn5],pmdecYSO[parsn5],s=5)
plt.show()
plt.quiver(raYSO[parsn5],decYSO[parsn5],pmraYSO[parsn5],pmdecYSO[parsn5])
plt.show()
mean_pmra = np.mean(pmraYSO[parsn10])
mean_pmdec = np.mean(pmdecYSO[parsn10])
print("Mean proper motion (Ra,Dec): ",mean_pmra,mean_pmdec)
median_pmra = np.median(pmraYSO[parsn10])
median_pmdec = np.median(pmdecYSO[parsn10])
print("Median proper motion (Ra,Dec): ",median_pmra,median_pmdec)
rpmraYSO = pmraYSO - median_pmra
rpmdecYSO = pmdecYSO - median_pmdec
pmmagYSO = np.zeros(len(YSO_match[good_YSO]))
pmmagYSO = np.power((rpmraYSO*rpmraYSO+rpmdecYSO*rpmdecYSO),0.5)
plt.quiver(raYSO[parsn5],decYSO[parsn5],rpmraYSO[parsn5],rpmdecYSO[parsn5],pmmagYSO[parsn5])
plt.colorbar()
plt.show()

In [ ]:
high_pmYSO = pmmagYSO[parsn5] > 8.
for i in range(0,len(high_pmYSO)):
    if(high_pmYSO[i]):
        j = YSO_match[i]
        print(Gra[j],Gdec[j],c2dYSOobj[i],rpmraYSO[i],rpmdecYSO[i],pmmagYSO[i])

In [ ]:
raSD = np.zeros(len(SD_match[good_SD]))
decSD = np.zeros(len(SD_match[good_SD]))
pmraSD = np.zeros(len(SD_match[good_SD]))
pmdecSD = np.zeros(len(SD_match[good_SD]))
pmra_eSD = np.zeros(len(SD_match[good_SD]))
pmdec_eSD = np.zeros(len(SD_match[good_SD]))
parSD = np.zeros(len(SD_match[good_SD]))
par_snSD = np.zeros(len(SD_match[good_SD]))
pmmagSD = np.zeros(len(SD_match[good_SD]))
nsd = 0
for i in range(0,len(SD_match)):
    if(SD_match[i] > -1):
        j = SD_match[i]
        if(1000./Gpar[j] > 350. and 1000./Gpar[j]<550):
            raSD[nsd] = Gra[j]
            decSD[nsd] = Gdec[j]
            pmraSD[nsd] = Gpmra[j]
            pmdecSD[nsd] = Gpmdec[j]
            parSD[nsd] = Gpar[j]
            par_snSD[nsd] = Gpar_sn[j]
            nsd = nsd + 1
print("Number of Star+dusts sources in distance interval: ",nsd)
parsn10SD = par_snSD > 10.
parsn5SD = par_snSD > 5
parsn20SD = par_snSD > 20
print("Number of SD with parallax > 5, 10, 20: ", len(par_snSD[parsn5SD]),len(par_snSD[parsn10SD]),len(par_snSD[parsn20SD]))
plt.scatter(raSD[parsn5SD],decSD[parsn5SD],s=5)
plt.show()
plt.scatter(pmraSD[parsn5SD],pmdecSD[parsn5SD],s=5)
plt.show()
plt.quiver(raSD[parsn5SD],decSD[parsn5SD],pmraSD[parsn5SD],pmdecSD[parsn5SD])
plt.show()
mean_pmraSD = np.mean(pmraSD[parsn10SD])
mean_pmdecSD = np.mean(pmdecSD[parsn10SD])
print("Mean proper motion (Ra,Dec): ",mean_pmraSD,mean_pmdecSD)
median_pmraSD = np.median(pmraSD[parsn10SD])
median_pmdecSD = np.median(pmdecSD[parsn10SD])
print("Median proper motion (Ra,Dec): ",median_pmra,median_pmdec)
rpmraSD = pmraSD - median_pmraSD
rpmdecSD = pmdecSD - median_pmdecSD
pmmagSD = np.zeros(len(SD_match[good_SD]))
pmmagSD = np.power((rpmraSD*rpmraSD+rpmdecSD*rpmdecSD),0.5)
plt.quiver(raSD[parsn5SD],decSD[parsn5SD],rpmraSD[parsn5SD],rpmdecSD[parsn5SD],pmmagSD[parsn5SD])
plt.colorbar()
plt.show()